In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
sentences = ['아르곤 용접시 사전에 환기를 합니다.', '어제 아르곤 용접 폭발 사고가 났습니다.', '도장 작업시 주변 정리를 철저히 합니다.']

# Without Qaunt

In [3]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

D:\GG_env\llm_env\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jongb\.cache\huggingface\hub\models--jhgan--ko-sroberta-multitask. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
embeddings = model.encode(sentences)
embeddings

array([[-6.0790128e-01, -1.1869144e-01,  3.7624347e-01, ...,
         6.1215335e-01,  4.0965819e-01, -1.7261702e-01],
       [-4.4415945e-01, -6.0619819e-01,  4.6051070e-01, ...,
         9.6765620e-04,  4.6226603e-01, -2.5306028e-01],
       [ 2.0911936e-01,  1.9398776e-01, -2.6798260e-01, ...,
         1.1331687e+00,  6.3240361e-01, -2.9789799e-01]], dtype=float32)

In [5]:
similarities = util.cos_sim(embeddings, embeddings) # compute similarity between sentence vectors
similarities

tensor([[1.0000, 0.4298, 0.5489],
        [0.4298, 1.0000, 0.1119],
        [0.5489, 0.1119, 1.0000]])

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

# Model Quantization

In [6]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel
from sentence_transformers import SentenceTransformer, util

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [9]:
model_id = "jhgan/ko-sroberta-multitask"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

In [ ]:
# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
similarities = util.cos_sim(embeddings, embeddings) # compute similarity between sentence vectors
similarities